In [ ]:
# cursor.close() 하지 않으면 DBserver에 trunc가 불가능 

In [52]:
import mysql.connector
import pandas as pd
import ipywidgets as widgets
from IPython.display import display

# explain : type_id == item_type / All columns except type_id are companies.

# Connect to the database DB 에 연결
conn = mysql.connector.connect(
    host="localhost",
    user="username",      # Mysql 아이디 (초기값 = root)
    password="userpassword!?",  # DB 초기 설정 비밀번호
    database="mydatabase"     # 생성 DB_name
)
cursor = conn.cursor()

# new DATA Input
def insert_item(type_id, TPC, SMC, KCC): 
    # Check if type_id is blank
    if not type_id.strip():
        print("Error: Primary key (type_id) cannot be blank.") # pk == blank (primary key == 공란 X)
        return
    
    # type_id를 제외한 나머지도 blank is ERROR
    if not TPC.strip() or not SMC.strip() or not KCC.strip():
        print("Error: Company names cannot be blank.")
        return
    
    sql = "INSERT INTO items (type, TPC, SMC, KCC) VALUES (%s, %s, %s, %s)"
    val = (type_id, TPC, SMC, KCC)
    cursor.execute(sql, val)
    conn.commit()
    print(cursor.rowcount, "Input Collect") 

# Function to read items
def read_items():
    cursor.execute("SELECT * FROM items")
    result = cursor.fetchall()
    df = pd.DataFrame(result, columns=['type', 'TPC', 'SMC', 'KCC'])
    display(df)

# Function to update an item
def update_item(type_id, new_TPC, new_SMC, new_KCC):
    sql = "UPDATE items SET TPC = %s, SMC = %s, KCC = %s WHERE type = %s"
    val = (new_TPC, new_SMC, new_KCC, type_id)
    cursor.execute(sql, val)
    conn.commit()
    print(cursor.rowcount, "record(s) affected")

# Function to delete an item
def delete_item(type_id):
    sql = "DELETE FROM items WHERE type = %s"
    val = (type_id,)
    cursor.execute(sql, val)
    conn.commit()
    print(cursor.rowcount, "record(s) deleted")

print('새로운 DATA 입력')

# Creating widgets for user input
type_input = widgets.Text(description="Type:")
TPC_input = widgets.Text(description="TPC:")
SMC_input = widgets.Text(description="SMC:")
KCC_input = widgets.Text(description="KCC:")
insert_button = widgets.Button(description="Insert")
display(type_input, TPC_input, SMC_input, KCC_input, insert_button)

# Function to handle insert button click event
def on_insert_button_clicked(b):
    type_id = type_input.value
    TPC = TPC_input.value
    SMC = SMC_input.value
    KCC = KCC_input.value
    insert_item(type_id, TPC, SMC, KCC)
    # Clear the input fields
    type_input.value = ""
    TPC_input.value = ""
    SMC_input.value = ""
    KCC_input.value = ""

# Assign the button click event
insert_button.on_click(on_insert_button_clicked)

print('DB 불러오기')

# Creating widgets for reading items
read_button = widgets.Button(description="Read Items Table")
display(read_button)

# Function to handle read button click event
def on_read_button_clicked(b):
    read_items()

# Assign the button click event
read_button.on_click(on_read_button_clicked)

print('DB DATA 수정하기')
print('미입력시 데이터 변경 없음')

# Creating widgets for updating an item
type_update_input = widgets.Text(description="Type:")
new_TPC_input = widgets.Text(description="New TPC:")
new_SMC_input = widgets.Text(description="New SMC:")
new_KCC_input = widgets.Text(description="New KCC:")
update_button = widgets.Button(description="Update")
display(type_update_input, new_TPC_input, new_SMC_input, new_KCC_input, update_button)


# Function to handle update button click event
def on_update_button_clicked(b):
    type_id = type_update_input.value
    new_TPC = new_TPC_input.value
    new_SMC = new_SMC_input.value
    new_KCC = new_KCC_input.value
    
    # Fetch current data from the database
    cursor.execute("SELECT TPC, SMC, KCC FROM items WHERE type = %s", (type_id,))
    result = cursor.fetchone()
    current_TPC, current_SMC, current_KCC = result
    
    # Check if any of the update fields are blank
    if not new_TPC.strip():
        new_TPC = current_TPC
    if not new_SMC.strip():
        new_SMC = current_SMC
    if not new_KCC.strip():
        new_KCC = current_KCC
    
    # Update item with validated data
    update_item(type_id, new_TPC, new_SMC, new_KCC)
    
    # Clear the input fields
    type_update_input.value = ""
    new_TPC_input.value = ""
    new_SMC_input.value = ""
    new_KCC_input.value = ""

# Assign the button click event
update_button.on_click(on_update_button_clicked)


print('DB DATA 삭제')
print('사용방법 : Type명을 입력하시면 해당 행이 지워집니다.')

# Creating widgets for deleting an item
type_delete_input = widgets.Text(description="type :")
delete_button = widgets.Button(description="Delete")
display(type_delete_input, delete_button)

# Function to handle delete button click event
def on_delete_button_clicked(b):
    type_id = type_delete_input.value
    delete_item(type_id)
    # Clear the input field
    type_delete_input.value = ""

# Assign the button click event
delete_button.on_click(on_delete_button_clicked)

# Close the connection when done
def close_connection():
    cursor.close()
    conn.close()


새로운 DATA 입력


Text(value='', description='Type:')

Text(value='', description='TPC:')

Text(value='', description='SMC:')

Text(value='', description='KCC:')

...

Button(description='Update', style=ButtonStyle())

DB DATA 삭제
사용방법 : Type명을 입력하시면 해당 행이 지워집니다.


Text(value='', description='type :')

Button(description='Delete', style=ButtonStyle())

,type,TPC,SMC,KCC
0,solid,fff,bbb,ccc


In [47]:
import mysql.connector
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
print('대응 표본 찾기')
# Connect to the database
conn = mysql.connector.connect(
    host="localhost",
    user="username",      # Mysql 아이디 (초기값 = root)
    password="userpassword!?",  # DB 초기 설정 비밀번호
    database="mydatabase"     # 생성 DB_name
)
cursor = conn.cursor()

# New widget for querying based on a specific column
search_column_input = widgets.Text(description="회사명:")
search_value_input = widgets.Text(description="품번:")
search_button = widgets.Button(description="검색")
output_area = widgets.Output()

display(search_column_input, search_value_input, search_button, output_area)

# Function to handle search button click event
def on_search_button_clicked(b):
    search_column = search_column_input.value
    search_value = search_value_input.value
    query = f"SELECT * FROM items WHERE {search_column} = %s"
    cursor.execute(query, (search_value,))
    result = cursor.fetchall()

    # Retrieve column names
    column_names = [i[0] for i in cursor.description]

    with output_area:
        output_area.clear_output()
        if result:
            # Create DataFrame
            df = pd.DataFrame(result, columns=column_names)
            # Display DataFrame
            display(df)
        else:
            print(f"No records found with {search_column} =", search_value)
    # Clear the input fields
    search_column_input.value = ""
    search_value_input.value = ""

# Assign the button click event
search_button.on_click(on_search_button_clicked)

# Close the connection when done
def close_connection():
    cursor.close()
    conn.close()


대응 표본 찾기


Text(value='', description='회사명:')

Text(value='', description='품번:')

Button(description='검색', style=ButtonStyle())

Output()

In [54]:
conn.close() # 실행 종료